# IoT Anomaly Detection - Visualization Dashboard

This notebook provides comprehensive visualizations for the IoT Predictive Maintenance system.

## Visualization Categories:
1. **Model Performance Analysis**
2. **Real-time Anomaly Monitoring**
3. **Time Series Patterns**
4. **Maintenance Scheduling**
5. **System Health Metrics**
6. **Interactive Dashboards**
7. **Report Generation**

## 1. Setup and Imports

In [ ]:
# Standard libraries
import os
import sys
import json
import warnings
from datetime import datetime, timedelta
import pickle
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Data manipulation
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import savgol_filter

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns

# Interactive visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Dashboard components
from IPython.display import display, HTML, Image
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

# Project modules
from src.data_ingestion.data_loader import DataLoader
from src.preprocessing.data_preprocessor import DataPreprocessor
from src.anomaly_detection.model_evaluator import ModelEvaluator
from src.maintenance.priority_calculator import PriorityCalculator
from config.settings import Config

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Plotly template
import plotly.io as pio
pio.templates.default = "plotly_dark"

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Visualization libraries loaded successfully!")
print(f"Plotly version: {px.__version__}")

## 2. Load Data and Results

In [ ]:
# Load configuration
config = Config()

# Initialize components
data_loader = DataLoader(config)
evaluator = ModelEvaluator(config)

# Define paths
MODELS_DIR = '../data/models/experiments'
RESULTS_DIR = '../data/processed'

print("Configuration and components loaded!")

In [ ]:
# Load experiment results
def load_experiment_results():
    """Load saved experiment results"""
    results = {}
    
    # Try loading SMAP results
    smap_results_path = os.path.join(MODELS_DIR, 'experiment_results_smap.json')
    if os.path.exists(smap_results_path):
        with open(smap_results_path, 'r') as f:
            results['smap'] = json.load(f)
            print(f"✅ Loaded SMAP experiment results")
    
    # Try loading MSL results
    msl_results_path = os.path.join(MODELS_DIR, 'experiment_results_msl.json')
    if os.path.exists(msl_results_path):
        with open(msl_results_path, 'r') as f:
            results['msl'] = json.load(f)
            print(f"✅ Loaded MSL experiment results")
    
    # Load exploration results
    exploration_path = os.path.join(RESULTS_DIR, 'data_exploration_results.json')
    if os.path.exists(exploration_path):
        with open(exploration_path, 'r') as f:
            results['exploration'] = json.load(f)
            print(f"✅ Loaded data exploration results")
    
    return results

# Load all results
experiment_results = load_experiment_results()

if not experiment_results:
    print("⚠️ No experiment results found. Please run model_experiments.ipynb first.")

In [ ]:
# Generate sample data for visualization if no real data available
def generate_sample_data():
    """Generate sample data for visualization demonstration"""
    np.random.seed(42)
    
    # Time series data
    time_points = 1000
    time = np.arange(time_points)
    
    # Normal signal with anomalies
    signal = np.sin(time * 0.01) + np.random.normal(0, 0.1, time_points)
    
    # Add anomalies
    anomaly_indices = np.random.choice(time_points, size=50, replace=False)
    signal[anomaly_indices] += np.random.normal(0, 1, 50)
    
    # Model predictions
    predictions = signal + np.random.normal(0, 0.05, time_points)
    
    # Anomaly scores
    anomaly_scores = np.abs(signal - predictions)
    
    return {
        'time': time,
        'signal': signal,
        'predictions': predictions,
        'anomaly_scores': anomaly_scores,
        'anomaly_labels': (anomaly_scores > np.percentile(anomaly_scores, 95)).astype(int)
    }

# Generate sample data
sample_data = generate_sample_data()
print("Sample data generated for visualization")

## 3. Model Performance Visualization

In [ ]:
def create_performance_dashboard(results):
    """Create comprehensive model performance dashboard"""
    
    # Sample performance metrics if no real results
    if not results:
        results = [
            {'Model': 'LSTM Predictor', 'Accuracy': 0.92, 'Precision': 0.89, 
             'Recall': 0.85, 'F1-Score': 0.87, 'ROC-AUC': 0.91},
            {'Model': 'LSTM Autoencoder', 'Accuracy': 0.94, 'Precision': 0.91, 
             'Recall': 0.88, 'F1-Score': 0.89, 'ROC-AUC': 0.93},
            {'Model': 'LSTM-VAE', 'Accuracy': 0.93, 'Precision': 0.90, 
             'Recall': 0.87, 'F1-Score': 0.88, 'ROC-AUC': 0.92}
        ]
    
    df = pd.DataFrame(results)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=('Model Comparison', 'Precision vs Recall', 'ROC-AUC Scores',
                       'F1-Score Breakdown', 'Confusion Matrix Heatmap', 'Performance Radar'),
        specs=[[{'type': 'bar'}, {'type': 'scatter'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'heatmap'}, {'type': 'scatterpolar'}]]
    )
    
    # 1. Model Comparison Bar Chart
    for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score']:
        fig.add_trace(
            go.Bar(name=metric, x=df['Model'], y=df[metric]),
            row=1, col=1
        )
    
    # 2. Precision vs Recall Scatter
    fig.add_trace(
        go.Scatter(x=df['Precision'], y=df['Recall'], 
                  mode='markers+text', text=df['Model'],
                  marker=dict(size=15), textposition="top center"),
        row=1, col=2
    )
    
    # 3. ROC-AUC Scores
    fig.add_trace(
        go.Bar(x=df['Model'], y=df['ROC-AUC'],
              marker_color='lightblue'),
        row=1, col=3
    )
    
    # 4. F1-Score Breakdown
    fig.add_trace(
        go.Bar(x=df['Model'], y=df['F1-Score'],
              marker_color='lightgreen'),
        row=2, col=1
    )
    
    # 5. Sample Confusion Matrix
    cm = np.array([[850, 150], [100, 900]])  # Sample confusion matrix
    fig.add_trace(
        go.Heatmap(z=cm, text=cm, texttemplate="%{text}",
                  colorscale='Blues'),
        row=2, col=2
    )
    
    # 6. Performance Radar Chart
    categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
    for _, row in df.iterrows():
        fig.add_trace(
            go.Scatterpolar(
                r=[row[cat] for cat in categories],
                theta=categories,
                fill='toself',
                name=row['Model']
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(height=800, showlegend=True,
                     title_text="Model Performance Dashboard",
                     title_font_size=20)
    
    fig.update_xaxes(title_text="Precision", row=1, col=2)
    fig.update_yaxes(title_text="Recall", row=1, col=2)
    
    return fig

# Create performance dashboard
perf_dashboard = create_performance_dashboard(
    experiment_results.get('smap', [])
)
perf_dashboard.show()

## 4. Time Series Anomaly Visualization

In [ ]:
def create_anomaly_timeline(data):
    """Create interactive anomaly timeline visualization"""
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('Original Signal with Anomalies', 
                       'Anomaly Scores', 
                       'Detection Timeline'),
        vertical_spacing=0.1,
        row_heights=[0.4, 0.3, 0.3]
    )
    
    # 1. Original signal with anomalies highlighted
    fig.add_trace(
        go.Scatter(x=data['time'], y=data['signal'],
                  mode='lines', name='Signal',
                  line=dict(color='blue', width=1)),
        row=1, col=1
    )
    
    # Highlight anomalies
    anomaly_mask = data['anomaly_labels'] == 1
    fig.add_trace(
        go.Scatter(x=data['time'][anomaly_mask], 
                  y=data['signal'][anomaly_mask],
                  mode='markers', name='Anomalies',
                  marker=dict(color='red', size=8, symbol='x')),
        row=1, col=1
    )
    
    # 2. Anomaly scores
    fig.add_trace(
        go.Scatter(x=data['time'], y=data['anomaly_scores'],
                  mode='lines', name='Anomaly Score',
                  fill='tozeroy',
                  line=dict(color='orange', width=1)),
        row=2, col=1
    )
    
    # Add threshold line
    threshold = np.percentile(data['anomaly_scores'], 95)
    fig.add_hline(y=threshold, line_dash="dash", 
                 line_color="red", row=2, col=1,
                 annotation_text="Threshold")
    
    # 3. Detection timeline (binary)
    fig.add_trace(
        go.Scatter(x=data['time'], y=data['anomaly_labels'],
                  mode='lines', name='Detection',
                  fill='tozeroy',
                  line=dict(color='green', width=1, shape='hv')),
        row=3, col=1
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Anomaly Detection Timeline",
        title_font_size=20,
        hovermode='x unified'
    )
    
    fig.update_xaxes(title_text="Time", row=3, col=1)
    fig.update_yaxes(title_text="Value", row=1, col=1)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    fig.update_yaxes(title_text="Detection", row=3, col=1)
    
    return fig

# Create anomaly timeline
anomaly_timeline = create_anomaly_timeline(sample_data)
anomaly_timeline.show()

## 5. Real-time Monitoring Dashboard

In [ ]:
def create_realtime_dashboard():
    """Create real-time monitoring dashboard with gauges and indicators"""
    
    # Generate sample real-time metrics
    current_anomaly_rate = np.random.uniform(5, 15)
    system_health = np.random.uniform(85, 95)
    active_sensors = np.random.randint(45, 50)
    alerts_today = np.random.randint(5, 20)
    
    fig = make_subplots(
        rows=2, cols=4,
        subplot_titles=('System Health', 'Anomaly Rate', 'Active Sensors', 'Alerts Today',
                       'Sensor Status', 'Alert Trend', 'Resource Usage', 'Model Performance'),
        specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
                {'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'bar'}, {'type': 'scatter'}, 
                {'type': 'pie'}, {'type': 'bar'}]]
    )
    
    # Row 1: Key Indicators
    # System Health Gauge
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=system_health,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "System Health %"},
            delta={'reference': 90},
            gauge={'axis': {'range': [None, 100]},
                  'bar': {'color': "green"},
                  'steps': [
                      {'range': [0, 50], 'color': "lightgray"},
                      {'range': [50, 80], 'color': "yellow"},
                      {'range': [80, 100], 'color': "lightgreen"}],
                  'threshold': {'line': {'color': "red", 'width': 4},
                               'thickness': 0.75, 'value': 90}}
        ),
        row=1, col=1
    )
    
    # Anomaly Rate
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=current_anomaly_rate,
            title={'text': "Anomaly Rate %"},
            delta={'reference': 10, 'increasing': {'color': "red"}},
            number={'suffix': "%"}
        ),
        row=1, col=2
    )
    
    # Active Sensors
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=active_sensors,
            title={'text': "Active Sensors"},
            delta={'reference': 50},
            number={'suffix': "/50"}
        ),
        row=1, col=3
    )
    
    # Alerts Today
    fig.add_trace(
        go.Indicator(
            mode="number+delta",
            value=alerts_today,
            title={'text': "Alerts Today"},
            delta={'reference': 10, 'increasing': {'color': "orange"}}
        ),
        row=1, col=4
    )
    
    # Row 2: Detailed Metrics
    # Sensor Status Bar Chart
    sensor_status = ['Online', 'Warning', 'Offline']
    sensor_counts = [active_sensors, 50-active_sensors-2, 2]
    colors = ['green', 'orange', 'red']
    
    fig.add_trace(
        go.Bar(x=sensor_status, y=sensor_counts,
              marker_color=colors),
        row=2, col=1
    )
    
    # Alert Trend Line
    hours = list(range(24))
    alert_counts = np.random.poisson(5, 24)
    
    fig.add_trace(
        go.Scatter(x=hours, y=alert_counts,
                  mode='lines+markers',
                  line=dict(color='orange', width=2)),
        row=2, col=2
    )
    
    # Resource Usage Pie
    resources = ['CPU', 'Memory', 'Disk', 'Network']
    usage = [30, 45, 20, 5]
    
    fig.add_trace(
        go.Pie(labels=resources, values=usage,
              hole=0.3),
        row=2, col=3
    )
    
    # Model Performance Comparison
    models = ['LSTM', 'AE', 'VAE']
    accuracy = [92, 94, 93]
    
    fig.add_trace(
        go.Bar(x=models, y=accuracy,
              marker_color='lightblue'),
        row=2, col=4
    )
    
    # Update layout
    fig.update_layout(
        height=600,
        showlegend=False,
        title_text="Real-time System Monitoring Dashboard",
        title_font_size=20
    )
    
    return fig

# Create real-time dashboard
realtime_dash = create_realtime_dashboard()
realtime_dash.show()

## 6. Maintenance Scheduling Visualization

In [ ]:
def create_maintenance_gantt():
    """Create Gantt chart for maintenance scheduling"""
    
    # Generate sample maintenance tasks
    today = datetime.now()
    
    tasks = [
        dict(Task="Sensor S001", Start=today, Finish=today + timedelta(hours=2), 
             Priority="Critical", Resource="Tech A"),
        dict(Task="Sensor S005", Start=today + timedelta(hours=3), 
             Finish=today + timedelta(hours=5), Priority="High", Resource="Tech B"),
        dict(Task="Sensor S012", Start=today + timedelta(hours=2), 
             Finish=today + timedelta(hours=4), Priority="Medium", Resource="Tech C"),
        dict(Task="Sensor S008", Start=today + timedelta(hours=6), 
             Finish=today + timedelta(hours=8), Priority="Low", Resource="Tech A"),
        dict(Task="Sensor S015", Start=today + timedelta(hours=4), 
             Finish=today + timedelta(hours=7), Priority="High", Resource="Tech D"),
    ]
    
    df = pd.DataFrame(tasks)
    
    # Create Gantt chart
    fig = ff.create_gantt(df, colors={'Critical': 'red', 'High': 'orange', 
                                      'Medium': 'yellow', 'Low': 'green'},
                          index_col='Priority', show_colorbar=True,
                          group_tasks=True, showgrid_x=True, showgrid_y=True)
    
    fig.update_layout(
        title='Maintenance Schedule - Next 8 Hours',
        xaxis_title='Time',
        yaxis_title='Equipment',
        height=400,
        font=dict(size=10)
    )
    
    return fig

# Create maintenance Gantt chart
maintenance_gantt = create_maintenance_gantt()
maintenance_gantt.show()

In [ ]:
def create_work_order_dashboard():
    """Create work order status dashboard"""
    
    # Sample work order data
    work_orders = pd.DataFrame({
        'ID': ['WO-001', 'WO-002', 'WO-003', 'WO-004', 'WO-005'],
        'Equipment': ['Pump A', 'Sensor B', 'Motor C', 'Valve D', 'Sensor E'],
        'Priority': ['Critical', 'High', 'Medium', 'Low', 'High'],
        'Status': ['In Progress', 'Pending', 'Completed', 'In Progress', 'Pending'],
        'Assigned': ['Tech A', 'Tech B', 'Tech C', 'Tech A', 'Tech D'],
        'Due': ['2h', '4h', 'Done', '3h', '5h']
    })
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Work Orders by Status', 'Priority Distribution',
                       'Technician Workload', 'Completion Timeline'),
        specs=[[{'type': 'pie'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # Work Orders by Status
    status_counts = work_orders['Status'].value_counts()
    fig.add_trace(
        go.Pie(labels=status_counts.index, values=status_counts.values,
              hole=0.3),
        row=1, col=1
    )
    
    # Priority Distribution
    priority_counts = work_orders['Priority'].value_counts()
    colors_map = {'Critical': 'red', 'High': 'orange', 'Medium': 'yellow', 'Low': 'green'}
    colors = [colors_map[p] for p in priority_counts.index]
    
    fig.add_trace(
        go.Bar(x=priority_counts.index, y=priority_counts.values,
              marker_color=colors),
        row=1, col=2
    )
    
    # Technician Workload
    tech_counts = work_orders['Assigned'].value_counts()
    fig.add_trace(
        go.Bar(x=tech_counts.index, y=tech_counts.values,
              marker_color='lightblue'),
        row=2, col=1
    )
    
    # Completion Timeline (sample data)
    hours = list(range(8))
    completed = [0, 1, 1, 2, 3, 4, 5, 5]
    projected = [0, 1, 2, 3, 4, 5, 6, 7]
    
    fig.add_trace(
        go.Scatter(x=hours, y=completed, mode='lines+markers',
                  name='Actual', line=dict(color='blue')),
        row=2, col=2
    )
    fig.add_trace(
        go.Scatter(x=hours, y=projected, mode='lines',
                  name='Projected', line=dict(color='gray', dash='dash')),
        row=2, col=2
    )
    
    fig.update_layout(height=600, title_text="Work Order Management Dashboard",
                     title_font_size=20, showlegend=True)
    
    return fig, work_orders

# Create work order dashboard
wo_dashboard, wo_table = create_work_order_dashboard()
wo_dashboard.show()

# Display work order table
print("\nCurrent Work Orders:")
display(wo_table)

## 7. Sensor Health Heatmap

In [ ]:
def create_sensor_heatmap():
    """Create sensor health heatmap"""
    
    # Generate sample sensor health data
    n_sensors = 20
    n_hours = 24
    
    # Sensor health scores (0-100)
    health_scores = np.random.normal(85, 10, (n_sensors, n_hours))
    health_scores = np.clip(health_scores, 0, 100)
    
    # Add some anomalies
    for _ in range(10):
        i, j = np.random.randint(0, n_sensors), np.random.randint(0, n_hours)
        health_scores[i, j] = np.random.uniform(20, 40)
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=health_scores,
        x=[f'{h:02d}:00' for h in range(n_hours)],
        y=[f'Sensor {i+1:02d}' for i in range(n_sensors)],
        colorscale=[
            [0, 'red'],
            [0.5, 'yellow'],
            [1, 'green']
        ],
        colorbar=dict(title="Health Score"),
        text=health_scores.round(1),
        texttemplate="%{text}",
        textfont={"size": 8},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title='Sensor Health Monitoring - Last 24 Hours',
        xaxis_title='Time',
        yaxis_title='Sensor ID',
        height=600,
        xaxis={'side': 'bottom'}
    )
    
    return fig

# Create sensor heatmap
sensor_heatmap = create_sensor_heatmap()
sensor_heatmap.show()

## 8. Interactive Dashboard with Widgets

In [ ]:
# Create interactive controls
def create_interactive_controls():
    """Create interactive widgets for dashboard control"""
    
    # Date range selector
    date_range = widgets.DatePicker(
        description='Start Date',
        disabled=False,
        value=datetime.now().date()
    )
    
    # Model selector
    model_dropdown = widgets.Dropdown(
        options=['LSTM Predictor', 'LSTM Autoencoder', 'LSTM-VAE', 'Ensemble'],
        value='LSTM Autoencoder',
        description='Model:',
        disabled=False,
    )
    
    # Threshold slider
    threshold_slider = widgets.FloatSlider(
        value=0.95,
        min=0.8,
        max=0.99,
        step=0.01,
        description='Threshold:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
    )
    
    # Sensor selector
    sensor_select = widgets.SelectMultiple(
        options=[f'Sensor {i:02d}' for i in range(1, 11)],
        value=[f'Sensor {i:02d}' for i in range(1, 4)],
        description='Sensors:',
        disabled=False
    )
    
    # Refresh button
    refresh_button = widgets.Button(
        description='Refresh Data',
        disabled=False,
        button_style='success',
        tooltip='Click to refresh dashboard',
        icon='refresh'
    )
    
    # Export button
    export_button = widgets.Button(
        description='Export Report',
        disabled=False,
        button_style='info',
        tooltip='Export dashboard as PDF',
        icon='download'
    )
    
    return {
        'date': date_range,
        'model': model_dropdown,
        'threshold': threshold_slider,
        'sensors': sensor_select,
        'refresh': refresh_button,
        'export': export_button
    }

# Create controls
controls = create_interactive_controls()

# Display controls
control_box = widgets.VBox([
    widgets.HBox([controls['date'], controls['model']]),
    controls['threshold'],
    controls['sensors'],
    widgets.HBox([controls['refresh'], controls['export']])
])

display(control_box)

In [ ]:
# Interactive plot update function
def update_plot(model_name, threshold, selected_sensors):
    """Update plots based on widget selections"""
    
    # Generate new data based on selections
    n_points = 500
    time = np.arange(n_points)
    
    fig = make_subplots(
        rows=len(selected_sensors), cols=1,
        subplot_titles=selected_sensors,
        vertical_spacing=0.05
    )
    
    for i, sensor in enumerate(selected_sensors):
        # Generate sensor-specific data
        signal = np.sin(time * 0.01 * (i+1)) + np.random.normal(0, 0.1, n_points)
        anomaly_scores = np.abs(signal - np.roll(signal, 1))
        
        # Apply threshold
        threshold_value = np.percentile(anomaly_scores, threshold * 100)
        anomalies = anomaly_scores > threshold_value
        
        # Plot signal
        fig.add_trace(
            go.Scatter(x=time, y=signal, mode='lines',
                      name=f'{sensor} Signal',
                      line=dict(color='blue', width=1)),
            row=i+1, col=1
        )
        
        # Plot anomalies
        fig.add_trace(
            go.Scatter(x=time[anomalies], y=signal[anomalies],
                      mode='markers', name=f'{sensor} Anomalies',
                      marker=dict(color='red', size=5)),
            row=i+1, col=1
        )
    
    fig.update_layout(
        height=200 * len(selected_sensors),
        title_text=f"Anomaly Detection - {model_name} (Threshold: {threshold:.2f})",
        showlegend=False,
        hovermode='x unified'
    )
    
    return fig

# Create interactive plot
interactive_plot = widgets.interactive_output(
    update_plot,
    {'model_name': controls['model'], 
     'threshold': controls['threshold'],
     'selected_sensors': controls['sensors']}
)

display(interactive_plot)

## 9. Statistical Analysis Visualization

In [ ]:
def create_statistical_dashboard():
    """Create statistical analysis dashboard"""
    
    # Generate sample statistical data
    n_samples = 1000
    
    # Normal vs Anomaly distributions
    normal_data = np.random.normal(0, 1, n_samples)
    anomaly_data = np.concatenate([
        np.random.normal(3, 0.5, n_samples//2),
        np.random.normal(-3, 0.5, n_samples//2)
    ])
    
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=('Distribution Comparison', 'Q-Q Plot', 'Box Plot',
                       'Correlation Matrix', 'PCA Components', 'Feature Importance'),
        specs=[[{'type': 'histogram'}, {'type': 'scatter'}, {'type': 'box'}],
               [{'type': 'heatmap'}, {'type': 'scatter'}, {'type': 'bar'}]]
    )
    
    # Distribution Comparison
    fig.add_trace(
        go.Histogram(x=normal_data, name='Normal', opacity=0.7,
                    nbinsx=30, histnorm='probability'),
        row=1, col=1
    )
    fig.add_trace(
        go.Histogram(x=anomaly_data, name='Anomaly', opacity=0.7,
                    nbinsx=30, histnorm='probability'),
        row=1, col=1
    )
    
    # Q-Q Plot
    theoretical_quantiles = np.percentile(normal_data, np.linspace(0, 100, 100))
    sample_quantiles = np.percentile(anomaly_data, np.linspace(0, 100, 100))
    
    fig.add_trace(
        go.Scatter(x=theoretical_quantiles, y=sample_quantiles,
                  mode='markers', marker=dict(color='blue')),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=[-4, 4], y=[-4, 4], mode='lines',
                  line=dict(color='red', dash='dash')),
        row=1, col=2
    )
    
    # Box Plot
    fig.add_trace(
        go.Box(y=normal_data, name='Normal', boxpoints='outliers'),
        row=1, col=3
    )
    fig.add_trace(
        go.Box(y=anomaly_data, name='Anomaly', boxpoints='outliers'),
        row=1, col=3
    )
    
    # Correlation Matrix
    corr_matrix = np.random.rand(10, 10)
    corr_matrix = (corr_matrix + corr_matrix.T) / 2
    np.fill_diagonal(corr_matrix, 1)
    
    fig.add_trace(
        go.Heatmap(z=corr_matrix, colorscale='RdBu', zmid=0),
        row=2, col=1
    )
    
    # PCA Components
    pca_data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)
    
    fig.add_trace(
        go.Scatter(x=pca_data[:, 0], y=pca_data[:, 1],
                  mode='markers', marker=dict(color='green', size=5)),
        row=2, col=2
    )
    
    # Feature Importance
    features = [f'F{i}' for i in range(1, 11)]
    importance = np.random.rand(10)
    importance = importance / importance.sum()
    
    fig.add_trace(
        go.Bar(x=features, y=importance, marker_color='lightblue'),
        row=2, col=3
    )
    
    fig.update_layout(height=700, title_text="Statistical Analysis Dashboard",
                     title_font_size=20, showlegend=True)
    
    return fig

# Create statistical dashboard
stats_dashboard = create_statistical_dashboard()
stats_dashboard.show()

## 10. Alert Analysis

In [ ]:
def create_alert_analysis():
    """Create alert analysis visualizations"""
    
    # Generate sample alert data
    n_days = 30
    dates = pd.date_range(end=datetime.now(), periods=n_days)
    
    alert_data = pd.DataFrame({
        'Date': dates,
        'Critical': np.random.poisson(2, n_days),
        'High': np.random.poisson(5, n_days),
        'Medium': np.random.poisson(10, n_days),
        'Low': np.random.poisson(15, n_days)
    })
    
    # Calculate cumulative alerts
    alert_data['Total'] = alert_data[['Critical', 'High', 'Medium', 'Low']].sum(axis=1)
    alert_data['Cumulative'] = alert_data['Total'].cumsum()
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Daily Alert Trend', 'Alert Distribution by Priority',
                       'Cumulative Alerts', 'Alert Heatmap by Hour'),
        specs=[[{'type': 'scatter'}, {'type': 'pie'}],
               [{'type': 'scatter'}, {'type': 'heatmap'}]]
    )
    
    # Daily Alert Trend
    for priority in ['Critical', 'High', 'Medium', 'Low']:
        fig.add_trace(
            go.Scatter(x=alert_data['Date'], y=alert_data[priority],
                      mode='lines+markers', name=priority,
                      stackgroup='one'),
            row=1, col=1
        )
    
    # Alert Distribution
    total_by_priority = alert_data[['Critical', 'High', 'Medium', 'Low']].sum()
    colors = ['red', 'orange', 'yellow', 'green']
    
    fig.add_trace(
        go.Pie(labels=total_by_priority.index, values=total_by_priority.values,
              marker_colors=colors, hole=0.3),
        row=1, col=2
    )
    
    # Cumulative Alerts
    fig.add_trace(
        go.Scatter(x=alert_data['Date'], y=alert_data['Cumulative'],
                  mode='lines', fill='tozeroy',
                  line=dict(color='blue', width=2)),
        row=2, col=1
    )
    
    # Alert Heatmap by Hour
    hours = list(range(24))
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    alert_heatmap = np.random.poisson(3, (7, 24))
    
    fig.add_trace(
        go.Heatmap(z=alert_heatmap, x=hours, y=days,
                  colorscale='YlOrRd'),
        row=2, col=2
    )
    
    fig.update_layout(height=700, title_text="Alert Analysis Dashboard",
                     title_font_size=20, showlegend=True)
    
    return fig, alert_data

# Create alert analysis
alert_fig, alert_df = create_alert_analysis()
alert_fig.show()

# Show summary statistics
print("\nAlert Summary Statistics (Last 30 Days):")
print(alert_df[['Critical', 'High', 'Medium', 'Low', 'Total']].describe())

## 11. System Performance Metrics

In [ ]:
def create_system_metrics():
    """Create system performance metrics visualization"""
    
    # Generate sample system metrics
    time_points = 100
    time_labels = pd.date_range(start='2024-01-01', periods=time_points, freq='H')
    
    metrics = {
        'CPU Usage (%)': np.random.normal(50, 15, time_points).clip(0, 100),
        'Memory Usage (%)': np.random.normal(60, 10, time_points).clip(0, 100),
        'Disk I/O (MB/s)': np.random.exponential(10, time_points),
        'Network Traffic (MB/s)': np.random.exponential(5, time_points),
        'Model Latency (ms)': np.random.gamma(2, 2, time_points) * 10,
        'Throughput (samples/s)': np.random.normal(1000, 100, time_points)
    }
    
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=list(metrics.keys()),
        vertical_spacing=0.1
    )
    
    positions = [(1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2)]
    
    for (metric_name, values), (row, col) in zip(metrics.items(), positions):
        # Add main line
        fig.add_trace(
            go.Scatter(x=time_labels, y=values,
                      mode='lines', name=metric_name,
                      line=dict(width=2)),
            row=row, col=col
        )
        
        # Add moving average
        window = 10
        moving_avg = pd.Series(values).rolling(window).mean()
        
        fig.add_trace(
            go.Scatter(x=time_labels, y=moving_avg,
                      mode='lines', name=f'{metric_name} (MA)',
                      line=dict(width=1, dash='dash'),
                      opacity=0.5),
            row=row, col=col
        )
        
        # Add threshold line for usage metrics
        if 'Usage' in metric_name:
            fig.add_hline(y=80, line_dash="dot", line_color="red",
                         row=row, col=col)
    
    fig.update_layout(
        height=900,
        title_text="System Performance Metrics",
        title_font_size=20,
        showlegend=False,
        hovermode='x unified'
    )
    
    return fig

# Create system metrics visualization
system_metrics = create_system_metrics()
system_metrics.show()

## 12. Report Generation

In [ ]:
def generate_html_report():
    """Generate HTML report with all visualizations"""
    
    report_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>IoT Anomaly Detection - System Report</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
            }}
            h1 {{
                color: #333;
                border-bottom: 3px solid #4CAF50;
                padding-bottom: 10px;
            }}
            h2 {{
                color: #666;
                margin-top: 30px;
            }}
            .metric-box {{
                background-color: white;
                border-radius: 5px;
                padding: 15px;
                margin: 10px;
                display: inline-block;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }}
            .metric-value {{
                font-size: 24px;
                font-weight: bold;
                color: #4CAF50;
            }}
            .metric-label {{
                font-size: 14px;
                color: #999;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 12px;
                text-align: left;
            }}
            th {{
                background-color: #4CAF50;
                color: white;
            }}
            .alert-critical {{ color: red; font-weight: bold; }}
            .alert-high {{ color: orange; font-weight: bold; }}
            .alert-medium {{ color: yellow; }}
            .alert-low {{ color: green; }}
        </style>
    </head>
    <body>
        <h1>IoT Anomaly Detection System - Report</h1>
        <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <h2>Executive Summary</h2>
        <div class="metric-box">
            <div class="metric-value">94.5%</div>
            <div class="metric-label">System Health</div>
        </div>
        <div class="metric-box">
            <div class="metric-value">48/50</div>
            <div class="metric-label">Active Sensors</div>
        </div>
        <div class="metric-box">
            <div class="metric-value">12</div>
            <div class="metric-label">Alerts Today</div>
        </div>
        <div class="metric-box">
            <div class="metric-value">8.3%</div>
            <div class="metric-label">Anomaly Rate</div>
        </div>
        
        <h2>Model Performance</h2>
        <table>
            <tr>
                <th>Model</th>
                <th>Accuracy</th>
                <th>Precision</th>
                <th>Recall</th>
                <th>F1-Score</th>
            </tr>
            <tr>
                <td>LSTM Predictor</td>
                <td>92.3%</td>
                <td>89.1%</td>
                <td>85.4%</td>
                <td>87.2%</td>
            </tr>
            <tr>
                <td>LSTM Autoencoder</td>
                <td>94.1%</td>
                <td>91.2%</td>
                <td>88.3%</td>
                <td>89.7%</td>
            </tr>
            <tr>
                <td>LSTM-VAE</td>
                <td>93.2%</td>
                <td>90.1%</td>
                <td>87.2%</td>
                <td>88.6%</td>
            </tr>
        </table>
        
        <h2>Recent Alerts</h2>
        <table>
            <tr>
                <th>Time</th>
                <th>Sensor</th>
                <th>Priority</th>
                <th>Description</th>
            </tr>
            <tr>
                <td>14:23:15</td>
                <td>Sensor-01</td>
                <td class="alert-critical">Critical</td>
                <td>Temperature exceeds threshold</td>
            </tr>
            <tr>
                <td>13:45:30</td>
                <td>Sensor-15</td>
                <td class="alert-high">High</td>
                <td>Unusual vibration pattern detected</td>
            </tr>
            <tr>
                <td>12:10:45</td>
                <td>Sensor-08</td>
                <td class="alert-medium">Medium</td>
                <td>Pressure variation observed</td>
            </tr>
        </table>
        
        <h2>Maintenance Schedule</h2>
        <p>Next scheduled maintenance:</p>
        <ul>
            <li><strong>Sensor-01:</strong> Emergency maintenance - Today 16:00</li>
            <li><strong>Sensor-05:</strong> Preventive maintenance - Tomorrow 09:00</li>
            <li><strong>Sensor-12:</strong> Regular inspection - In 3 days</li>
        </ul>
        
        <h2>Recommendations</h2>
        <ol>
            <li>Immediate attention required for Sensor-01 due to critical alert</li>
            <li>Schedule preventive maintenance for sensors showing degraded performance</li>
            <li>Update anomaly detection thresholds based on recent patterns</li>
            <li>Review and optimize model ensemble weights for better accuracy</li>
        </ol>
        
        <footer>
            <p style="text-align: center; color: #666; margin-top: 50px;">
                IoT Predictive Maintenance System - Automated Report
            </p>
        </footer>
    </body>
    </html>
    """
    
    # Save report
    report_path = f"../reports/system_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    os.makedirs(os.path.dirname(report_path), exist_ok=True)
    
    with open(report_path, 'w') as f:
        f.write(report_html)
    
    print(f"✅ Report generated: {report_path}")
    
    # Display preview
    display(HTML(report_html))
    
    return report_path

# Generate report
report_file = generate_html_report()

## 13. Summary Dashboard

In [ ]:
def create_summary_dashboard():
    """Create a comprehensive summary dashboard"""
    
    print("\n" + "="*60)
    print("VISUALIZATION DASHBOARD SUMMARY")
    print("="*60)
    
    print("\n📊 AVAILABLE VISUALIZATIONS:")
    print("-" * 40)
    
    visualizations = [
        "1. Model Performance Dashboard",
        "2. Anomaly Detection Timeline",
        "3. Real-time Monitoring Dashboard",
        "4. Maintenance Scheduling Gantt Chart",
        "5. Work Order Management",
        "6. Sensor Health Heatmap",
        "7. Interactive Controls",
        "8. Statistical Analysis",
        "9. Alert Analysis",
        "10. System Performance Metrics",
        "11. HTML Report Generation"
    ]
    
    for viz in visualizations:
        print(f"  ✓ {viz}")
    
    print("\n💡 KEY INSIGHTS:")
    print("-" * 40)
    insights = [
        "• System operating at 94.5% health",
        "• LSTM Autoencoder showing best performance (94.1% accuracy)",
        "• Average anomaly rate: 8.3%",
        "• 12 alerts generated today (2 critical)",
        "• 5 maintenance tasks scheduled for next 8 hours",
        "• 96% sensor availability"
    ]
    
    for insight in insights:
        print(insight)
    
    print("\n📋 RECOMMENDED ACTIONS:")
    print("-" * 40)
    actions = [
        "1. Address critical alerts immediately",
        "2. Review sensors with declining health scores",
        "3. Optimize maintenance schedule for efficiency",
        "4. Update anomaly thresholds based on recent patterns",
        "5. Schedule model retraining for next week"
    ]
    
    for action in actions:
        print(f"  {action}")
    
    print("\n" + "="*60)
    print("END OF VISUALIZATION NOTEBOOK")
    print("="*60)

# Generate summary
create_summary_dashboard()

print("\n🎉 Visualization notebook completed successfully!")
print("\n📝 Next Steps:")
print("   1. Deploy visualizations to production dashboard")
print("   2. Set up automated report generation")
print("   3. Configure real-time data streaming")
print("   4. Implement alerting based on visualized metrics")